In [ ]:
import fiona
import geopandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyproj
from colour import HSL_to_RGB
from inflection import parameterize

from constants import OUTPUT_DATA, RAW_DATA, cols, df_types

- https://en.wikipedia.org/wiki/List_of_districts_of_Nepal
- https://mapshaper.org/
- https://geopandas.org/en/stable/docs/user_guide/mapping.html
- https://geopandas.org/en/stable/docs/user_guide/projections.html#re-projecting
- https://proj.org/operations/projections/eqearth.html
- https://geopandas.org/en/stable/docs/user_guide/aggregation_with_dissolve.html
- https://geopandas.org/en/stable/docs/user_guide/io.html#writing-spatial-data
- https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.savefig.html
- https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.margins.html
- https://colour.readthedocs.io/en/latest/generated/colour.HSL_to_RGB.html#colour.HSL_to_RGB
- https://matplotlib.org/devdocs/gallery/subplots_axes_and_figures/figure_size_units.html#figure-size-in-pixel
- https://matplotlib.org/stable/tutorials/colors/colors.html

In [ ]:
DISTRICT_MAP = "raw/nepal/district/npl_admbnda_districts_nd_20201117.shp"

In [ ]:
gdf = geopandas.read_file(DISTRICT_MAP)

In [ ]:
gdf.head()

In [ ]:
gdf.shape

In [ ]:
gdf.crs

In [ ]:
gdf.plot(figsize=(15, 10))

In [ ]:
equal_earth_code = pyproj.CRS("+proj=eqearth").to_epsg()
equal_earth_code

In [ ]:
ee_gdf = gdf.to_crs(epsg=equal_earth_code)

In [ ]:
ee_gdf.crs

In [ ]:
ee_gdf.plot(figsize=(15, 10))

In [ ]:
ee_gdf.head()

In [ ]:
ee_gdf.tail()

In [ ]:
ee_gdf[ee_gdf["DIST_EN"].str.contains("Nawalparasi")]

In [ ]:
ee_gdf[ee_gdf["DIST_EN"].str.contains("Rukum")]

In [ ]:
# df = pd.read_excel(
#     RAW_DATA,
#     index_col=None,
#     sheet_name="Worksheet",
#     verbose=True,
#     skipfooter=1,
#     dtype=df_types,
#     usecols=cols,
# )

# df.columns = df.columns.str.strip()

df = pd.read_json(OUTPUT_DATA, convert_dates=False, orient="records")
df.head()

In [ ]:
# df["Province Name"].value_counts(dropna=False)
df["province_name"].value_counts(dropna=False)

In [ ]:
# df["Province Name"].nunique()
df["province_name"].nunique()

In [ ]:
# df["District"].value_counts(dropna=False)
df["district"].value_counts(dropna=False)

In [ ]:
# df["District"].nunique()
df["district"].nunique()

In [ ]:
# df["District"].sort_values().unique().tolist()

In [ ]:
# ee_gdf["DIST_EN"].to_list()

In [ ]:
# df.loc[~df["District"].isin(ee_gdf["DIST_EN"]), "District"].value_counts()
df.loc[~df["district"].isin(ee_gdf["DIST_EN"]), "district"].value_counts()

In [ ]:
# df.query("District == 'Dang Deokhuri'")

> Source: https://en.wikipedia.org/wiki/List_of_districts_of_Nepal and https://en.wikipedia.org/wiki/Dang_District,_Nepal

**Map data**

- ~Nawalparasi East + Nawalparasi West -> Nawalparasi~
- Makawanpur -> Makwanpur
- Chitawan -> Chitawan
- Kabhrepalanchok -> Kavrepalanchok
- ~Rukum East + Rukum West -> Rukum~

**Raw data** (`prep.ipynb`)

- Dang Deokhuri -> Dang
- Nawalparasi -> Nawalparasi East or Nawalparasi West
- Rukum -> Rukum East or Rukum West

In [ ]:
# df["Province Name"].sort_values().unique().tolist()
df["province_name"].sort_values().unique().tolist()

In [ ]:
ee_gdf["ADM1_EN"].sort_values().unique().tolist()

**Map data**

- Sudurpaschim -> Sudurpashchim

In [ ]:
province_map = {
    "Bagmati": "Bagmati Province",
    "Gandaki": "Gandaki Province",
    "Karnali": "Karnali Province",
    "Lumbini": "Lumbini Province",
    "Province 1": "Province No. 1",
    "Province 2": "Province No. 2",
    "Sudurpaschim": "Sudurpashchim Province",
}

ee_gdf["ADM1_EN"] = ee_gdf["ADM1_EN"].replace(province_map)

In [ ]:
district_map = {
    # "Nawalparasi East": "Nawalparasi",
    # "Nawalparasi West": "Nawalparasi",
    "Makawanpur": "Makwanpur",
    "Chitawan": "Chitawan",
    "Kabhrepalanchok": "Kavrepalanchok",
    # "Rukum East": "Rukum",
    # "Rukum West": "Rukum",
}

ee_gdf["DIST_EN"] = ee_gdf["DIST_EN"].replace(district_map)

In [ ]:
# ee_gdf["DIST_EN"].unique()
# ee_gdf.head()
# ee_gdf.crs

In [ ]:
ee_gdf.query("DIST_EN == 'Rukum'").plot()

In [ ]:
# hsl(252, 4%, 57.3%)
# hsl(260, 25%, 11%)
# hsl(252, 4%, 44.8%)

bg_color = HSL_to_RGB([252 / 360, 4 / 100, 57.3 / 100])

main_color = HSL_to_RGB([260 / 360, 25 / 100, 11 / 100])

edge_color = HSL_to_RGB([252 / 360, 4 / 100, 44.8 / 100])

In [ ]:
print(bg_color, main_color, edge_color, sep="\n")

In [ ]:
ee_gdf.query("ADM1_EN == 'Bagmati Province'").sample()

In [ ]:
def generate_minimaps_province(df, province_name):
    pdf = df.query("ADM1_EN == @province_name")
    # colors = [bg_color] * pdf.shape[0]
    colors = [main_color, bg_color]

    for i, district in enumerate(pdf["DIST_EN"].unique()):
        output = parameterize(f"{province_name}-{district}", separator="_")

        print(i, district, output, sep=" | ")

        # tmp_colors = colors.copy()
        # tmp_colors[i] = main_color

        tmp_pdf = pdf.copy()

        tmp_pdf["dissolve"] = np.where(
            tmp_pdf["DIST_EN"] != district,
            "Y",
            "N",
        )

        tmp_pdf = tmp_pdf.dissolve(by="dissolve", sort=True)

        tmp_pdf.plot(color=colors, aspect="auto", edgecolor="white", linewidth=0.25)

        plt.axis("off")
        plt.margins(0)

        plt.savefig(
            f"raw/minimaps/{output}.svg",
            format="svg",
            transparent=True,
            bbox_inches="tight",
            pad_inches=0,
            # backend="svg"
            backend="cairo",
        )

        plt.close()

In [ ]:
for province in ee_gdf["ADM1_EN"].unique():
    generate_minimaps_province(ee_gdf, province)

---